# Echo Transfer File Processor for ZIKV NS2B-NS3

This notebook processes the ZIKV NS2B-NS3 sample preparation data to create Echo transfer CSV files batched by plate-ID. Each output file contains the necessary columns for Echo liquid handling system transfers.

## Process Overview:
1. Load sample prep data from CSV
2. Group transfers by plate-ID
3. Generate destination wells (A1 through P24 for 384-well plates)
4. Create individual transfer files for each plate
5. Generate summary report

## 1. Import Required Libraries

In [21]:
import pandas as pd
import re
from pathlib import Path

print("Libraries imported successfully!")

Libraries imported successfully!


## 2. Configuration Setup

In [22]:
# Configuration settings
PLATENAME = "Zika-NS2B-NS3"
PLATETYPE = "creoptix"
INPUT_FILE = "ZIKV NS2B-NS3 sample prep for Echo.csv"
OUTPUT_DIR = "transfer-files"  # Current directory

# File paths
current_dir = Path(__file__).parent if '__file__' in globals() else Path('.')
input_path = current_dir / INPUT_FILE
output_dir = Path(OUTPUT_DIR)

print(f"Configuration loaded:")
print(f"  Plate Name: {PLATENAME}")
print(f"  Plate Type: {PLATETYPE}")
print(f"  Input File: {input_path}")
print(f"  Output Directory: {output_dir}")

# Ensure output directory exists
output_dir.mkdir(exist_ok=True)

Configuration loaded:
  Plate Name: Zika-NS2B-NS3
  Plate Type: creoptix
  Input File: ZIKV NS2B-NS3 sample prep for Echo.csv
  Output Directory: transfer-files


## 3. Load Sample Prep Data

In [23]:
# Load the sample prep data
try:
    df = pd.read_csv(input_path)
    print(f"Successfully loaded {len(df)} rows of data")
    print(f"Columns: {list(df.columns)}")
    
    # Display basic info about the data
    print(f"\nUnique plates found: {df['plate_ID'].nunique()}")
    print(f"Plate IDs: {sorted(df['plate_ID'].unique())}")
    
    # Show first few rows
    print(f"\nFirst 5 rows:")
    print(df.head())
    
except FileNotFoundError:
    print(f"Error: Could not find input file at {input_path}")
    print("Please ensure the file exists in the current directory.")
except Exception as e:
    print(f"Error loading data: {e}")

Successfully loaded 3360 rows of data
Columns: ['plate_ID', 'Source Well', 'ASAP_batch_ID', 'Vol from stock nl']

Unique plates found: 87
Plate IDs: ['1530852-Y4-110', '1530852-Y4-113', '1530852-Y4-116', '1530852-Y4-118', '1530852-Y4-122', '1530852-Y4-123', '1530852-Y4-127', '1530852-Y4-128', '1530852-Y4-132', '1530852-Y4-135', '1530852-Y4-137', '1530852-Y4-138', '1530852-Y4-142', '1530852-Y4-143', '1530852-Y4-147', '1530852-Y4-148', '1530852-Y4-152', '1530852-Y4-154', '1530852-Y4-158', '1530852-Y4-162', '1530852-Y4-166', '1530852-Y4-167', '1530852-Y4-170', '1530852-Y4-174', '1530852-Y4-179', '1530852-Y4-183', '1530852-Y4-187', '1530852-Y4-192', '1530852-Y4-195', '1530852-Y4-199', '1530852-Y4-200', '1530852-Y4-205', '1530852-Y4-206', '1530852-Y4-210', '1530852-Y4-213', '1530852-Y4-214', '1530852-Y4-217', '1530852-Y4-218', '1530852-Y4-221', '1530852-Y4-224', '1530852-Y4-228', '1530852-Y4-231', '1530852-Y4-235', '1530852-Y4-238', '1530852-Y4-242', '1530852-Y4-243', '1530852-Y4-246', '153

## 4. Define Helper Functions

In [24]:
def generate_plate_batch_name(plate_number):
    """Generate plate batch name using the format: platename + platetype + _p## """
    return f"{PLATENAME}_{PLATETYPE}_p{plate_number:02d}"

def normalize_well_format(well):
    """Convert well format to standard format (e.g., A05 -> A5, AC05 -> AC5)"""
    if pd.isna(well):
        return well
    
    # Extract letter part and number part
    well_str = str(well).strip()
    
    # Handle multi-letter rows (like AA, AB, AC, etc.)
    match = re.match(r'([A-Z]+)(\d+)', well_str.upper())
    if match:
        letters, numbers = match.groups()
        # Remove leading zeros from numbers
        return f"{letters}{int(numbers)}"
    
    return well_str

def generate_destination_wells(num_wells):
    """Generate destination wells in order from A1 through P24 for 384-well plates"""
    wells = []
    rows = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P']
    cols = list(range(1, 25))  # 1 to 24
    
    # Generate wells in column-first order (A1, B1, C1, ... P1, A2, B2, ...)
    for col in cols:
        for row in rows:
            wells.append(f"{row}{col}")
            if len(wells) >= num_wells:
                return wells[:num_wells]
    
    # If we need more than 384 wells, this would be an error
    if num_wells > len(wells):
        print(f"Warning: Requested {num_wells} wells but 384-well plate only has {len(wells)} wells")
    
    return wells[:num_wells]

# Test the functions
print("Testing helper functions:")
print(f"Plate batch name example: {generate_plate_batch_name(1)}")
print(f"Well format examples: {normalize_well_format('A05')} -> {normalize_well_format('AC05')}")
print(f"First 10 destination wells: {generate_destination_wells(10)}")

Testing helper functions:
Plate batch name example: Zika-NS2B-NS3_creoptix_p01
Well format examples: A5 -> AC5
First 10 destination wells: ['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1', 'J1']


## 5. Process Each Plate and Create Transfer Files

In [25]:
# Main processing loop
summary_data = []
plate_counter = 1

# Group by plate_ID and process each plate
unique_plates = sorted(df['plate_ID'].unique())
print(f"Processing {len(unique_plates)} unique plates...")
print("=" * 60)

for plate_id in unique_plates:
    # Filter data for this plate
    plate_data = df[df['plate_ID'] == plate_id].copy()
    
    # Generate plate batch name
    plate_batch_name = generate_plate_batch_name(plate_counter)
    
    # Create destination wells
    num_transfers = len(plate_data)
    destination_wells = generate_destination_wells(num_transfers)
    
    # Create transfer dataframe
    transfer_df = pd.DataFrame({
        'PlateBatch': [plate_batch_name] * num_transfers,
        'SrcWell': [normalize_well_format(well) for well in plate_data['Source Well']],
        'Destination well': destination_wells,
        'XferVol': plate_data['Vol from stock nl'].values
    })
    
    # Create output filename
    output_filename = f"plate-{plate_id}-transfers.csv"
    output_path = output_dir / output_filename
    
    # Save to CSV
    transfer_df.to_csv(output_path, index=False)
    
    # Store summary information
    summary_data.append({
        'plate_id': plate_id,
        'plate_batch_name': plate_batch_name,
        'num_transfers': num_transfers,
        'output_file': output_filename
    })
    
    # Print progress
    print(f"✓ Plate {plate_counter:2d}: {plate_id}")
    print(f"  - Plate batch: {plate_batch_name}")
    print(f"  - Transfers: {num_transfers}")
    print(f"  - Output file: {output_filename}")
    print()
    
    plate_counter += 1

print(f"Processing complete! Created {len(summary_data)} transfer files.")

Processing 87 unique plates...
✓ Plate  1: 1530852-Y4-110
  - Plate batch: Zika-NS2B-NS3_creoptix_p01
  - Transfers: 2
  - Output file: plate-1530852-Y4-110-transfers.csv

✓ Plate  2: 1530852-Y4-113
  - Plate batch: Zika-NS2B-NS3_creoptix_p02
  - Transfers: 3
  - Output file: plate-1530852-Y4-113-transfers.csv

✓ Plate  3: 1530852-Y4-116
  - Plate batch: Zika-NS2B-NS3_creoptix_p03
  - Transfers: 1
  - Output file: plate-1530852-Y4-116-transfers.csv

✓ Plate  4: 1530852-Y4-118
  - Plate batch: Zika-NS2B-NS3_creoptix_p04
  - Transfers: 2
  - Output file: plate-1530852-Y4-118-transfers.csv

✓ Plate  5: 1530852-Y4-122
  - Plate batch: Zika-NS2B-NS3_creoptix_p05
  - Transfers: 8
  - Output file: plate-1530852-Y4-122-transfers.csv

✓ Plate  6: 1530852-Y4-123
  - Plate batch: Zika-NS2B-NS3_creoptix_p06
  - Transfers: 2
  - Output file: plate-1530852-Y4-123-transfers.csv

✓ Plate  7: 1530852-Y4-127
  - Plate batch: Zika-NS2B-NS3_creoptix_p07
  - Transfers: 14
  - Output file: plate-1530852-Y4-

## 6. Write Summary Report to Text File

In [26]:
# Write the final summary report to a text file
from datetime import datetime

# Create the report content
report_lines = []
report_lines.append("=" * 80)
report_lines.append("FINAL SUMMARY REPORT")
report_lines.append("Echo Transfer File Processing")
report_lines.append(f"Generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
report_lines.append("=" * 80)

report_lines.append(f"\nTotal plates processed: {len(summary_df)}")
report_lines.append(f"Total transfers created: {summary_df['num_transfers'].sum()}")

report_lines.append(f"\nDetailed breakdown:")
report_lines.append("-" * 80)
report_lines.append(f"{'Source plate name':<20} {'Destination plate name':<30} {'Transfers':<10} {'Output File'}")
report_lines.append("-" * 80)

for _, row in summary_df.iterrows():
    report_lines.append(f"{row['plate_id']:<20} {row['plate_batch_name']:<30} {row['num_transfers']:<10} {row['output_file']}")

report_lines.append("-" * 80)

# Statistics
report_lines.append(f"\nTransfer Statistics:")
report_lines.append(f"  - Minimum transfers per plate: {summary_df['num_transfers'].min()}")
report_lines.append(f"  - Maximum transfers per plate: {summary_df['num_transfers'].max()}")
report_lines.append(f"  - Average transfers per plate: {summary_df['num_transfers'].mean():.1f}")

report_lines.append(f"\nAll transfer files have been saved in: {output_dir.absolute()}")

# Write to text file
report_filename = "Echo_Transfer_Summary_Report.txt"
report_path = output_dir / report_filename

with open(report_path, 'w') as f:
    f.write('\n'.join(report_lines))

print(f"✓ Summary report written to: {report_path}")
print(f"✓ Report contains {len(report_lines)} lines")

# Also display the content
print(f"\nReport content preview:")
print('\n'.join(report_lines))

✓ Summary report written to: transfer-files/Echo_Transfer_Summary_Report.txt
✓ Report contains 104 lines

Report content preview:
FINAL SUMMARY REPORT
Echo Transfer File Processing
Generated on: 2026-02-03 18:02:59

Total plates processed: 87
Total transfers created: 3360

Detailed breakdown:
--------------------------------------------------------------------------------
Source plate name    Destination plate name         Transfers  Output File
--------------------------------------------------------------------------------
1530852-Y4-110       Zika-NS2B-NS3_creoptix_p01     2          plate-1530852-Y4-110-transfers.csv
1530852-Y4-113       Zika-NS2B-NS3_creoptix_p02     3          plate-1530852-Y4-113-transfers.csv
1530852-Y4-116       Zika-NS2B-NS3_creoptix_p03     1          plate-1530852-Y4-116-transfers.csv
1530852-Y4-118       Zika-NS2B-NS3_creoptix_p04     2          plate-1530852-Y4-118-transfers.csv
1530852-Y4-122       Zika-NS2B-NS3_creoptix_p05     8          plate-1530852-